In [5]:
import wikipedia

wikipedia.set_lang("ko")
page = wikipedia.page("서버 전송 이벤트")
raw_text = page.content

print(raw_text[:5000])  # 일부 미리보기

서버 전송 이벤트(Server-sent events, SSE)는 클라이언트가 HTTP 연결을 통해 서버로부터 자동 업데이트를 수신할 수 있도록 하는 서버 푸시 기술이며, 초기 클라이언트 연결이 설정된 후 서버가 클라이언트를 향한 데이터 전송을 시작하는 방법을 설명한다. 이는 일반적으로 브라우저 클라이언트에 메시지 업데이트 또는 지속적인 데이터 스트림을 보내는 데 사용되며 클라이언트가 이벤트 스트림을 수신하기 위해 특정 URL을 요청하는 EventSource라는 자바스크립트 API를 통해 기본 브라우저 간 스트리밍을 향상시키도록 설계되었다. EventSource API는 WHATWG에 의해 HTML Living Standard의 일부로 표준화되었다. SSE의 미디어 유형은 text/event-stream이다.
파이어폭스 6+, 구글 크롬 6+, 오페라 11.5+, 사파리 5+, 마이크로소프트 엣지 79+ 등 모든 최신 브라우저는 서버에서 전송되는 이벤트를 지원한다.


== 역사 ==
SSE 메커니즘은 2004년에 시작된 "WHATWG 웹 애플리케이션 1.0" 제안의 일부로 이안 힉슨(Ian Hickson)에 의해 처음 지정되었다. 2006년 9월 오페라 웹 브라우저는 "서버 전송 이벤트"라는 기능으로 실험적 기술을 구현했다.


== 예시 ==


== 같이 보기 ==
푸시 기법
코멧 (프로그래밍)


== 각주 ==


== 외부 링크 ==
Server-Sent Events. HTML Living Standard.
HTML5 Server-push Technologies, Part 1. Introduction into HTML5 Server-push Technologies. Part 1 covers ServerSent Events.
Using Server-Sent Events. A concise example of how to use server-sent events, on the Mozilla Developer Network.
EventSource refer

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_text(raw_text)
print(f"총 {len(chunks)}개의 청크가 생성됨.")

총 3개의 청크가 생성됨.


In [7]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("BAAI/bge-m3", device="cpu")
vectors = embedding_model.encode(chunks)


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

client = QdrantClient(host="localhost", port=6333)

# 예: backend 컬렉션 만들기 (처음 1회만)
client.recreate_collection(
    collection_name="backend",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

# 벡터 업서트 (ID 중복 방지를 위해 hash 사용)
points = [
    PointStruct(
        id=abs(hash(chunk)),
        vector=vec.tolist(),
        payload={"text": chunk, "source": "backend", "title": "서버 전송 이벤트"}
    )
    for chunk, vec in zip(chunks, vectors)
]

client.upsert(collection_name="backend", points=points)
